## Overview

This example demonstrates an experimental NLP-based drift detection algorithm. It uses the "Civil Comments" dataset ([link](https://github.com/p-lambda/wilds/blob/main/wilds/datasets/civilcomments_dataset.py) to a Python loading script with additional details/links) from the `wilds` library, which contains online comments meant to be used in toxicity classification problems.

This example and the experimental modules often pull directly and indirectly from [`alibi-detect`](https://github.com/SeldonIO/alibi-detect/tree/master) and its own [example(s)](https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_text_imdb.html).

## Notes

This code is experimental, and has notable issues:
- transform functions are very slow, on even moderate batch sizes
- detector design is not generalized, and may not work on streaming problems, or with data representations of different types/shapes
- some warnings below are not addressed

## Imports

Code (transforms, alarm, detector) is pulled from the experimental module in `menelaus`, which is live but not fully tested. Note that commented code shows `wilds` modules being used to access and save the dataset to disk, but are excluded to save time. The example hence assumes the dataset is locally available.

In [1]:
import pandas as pd
# from wilds import get_dataset

from menelaus.experimental.transform import auto_tokenize, extract_embedding, uae_reduce_dimension
from menelaus.experimental.detector import Detector
from menelaus.experimental.alarm import KolmogorovSmirnovAlarm

c:\Users\ASRIVASTAVA\Documents\repos\menelaus\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

Since some of the experimental modules are not very performant, the dataset is loaded and then limited to the first 300 data points (comments), which are split into three sequential batches of 100.

In [2]:
# civil comments
# dataset_civil = get_dataset(dataset="civilcomments", download=True, root_dir="./wilds_datasets")
dataset_civil = pd.read_csv('wilds_datasets/civilcomments_v1.0/all_data_with_identities.csv')
dataset_civil = dataset_civil['comment_text'][:300].tolist()
batch1 = dataset_civil[:100]
batch2 = dataset_civil[100:200]
batch3 = dataset_civil[200:300]

## Transforms Pipeline

The major step is to initialize the transform functions that will be applied to the comments, to turn them into detector-compatible representations. 

First, the comments must be tokenized:
- set up an `AutoTokenizer` model from the `transformers` library with a convenience function, by specifying the desired model name and other arguments
- the convenience function lets the configured tokenizer be called repeatedly, using batch 1 as the training data

Then, the tokens must be made into embeddings:
- an initial transform function uses a `transformers` model to extract embeddings from given tokens
- the subsequent transform function reduces the dimension via an `UntrainedAutoEncoder` to a manageable size

In [6]:
# tokens 
tokenizer = auto_tokenize(model_name='bert-base-cased', pad_to_max_length=True, return_tensors='tf')
tokens = tokenizer(data=batch1)

# embedding (TODO abstract this layers line)
layers = [-_ for _ in range(1, 8 + 1)]
embedder = extract_embedding(model_name='bert-base-cased', embedding_type='hidden_state', layers=layers)

# dimension reduction via Untrained AutoEncoder
uae_reduce = uae_reduce_dimension(enc_dim=32)

c:\Users\ASRIVASTAVA\Documents\repos\menelaus\venv\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Detector Setup

Next a detector is setup. First, a `KolmogorovSmirnovAlarm` is initialized with default settings. When the amount of columns (which reject the null KS test hypothesis) exceeds the default ratio (0.25), this alarm will indicate drift has occurred. 

Then the detector is constructed. It is given the initialized alarm, and the ordered list of transforms configured above. The detector is then made to step through each available batch, and its state is printed as output. Note that the first batch establishes the reference data, the second establishes the test data, and the third will require recalibration (test is combined into reference) if drift is detected.

In [8]:
# detector + set reference
ks_alarm = KolmogorovSmirnovAlarm()
detector = Detector(alarm=ks_alarm, transforms=[tokenizer, embedder, uae_reduce])
detector.step(batch1)
print(f"\nState after initial batch: {detector.state}\n")

# detector + add test (copy reference)  
detector.step(batch2)
print(f"\nState after test batch: {detector.state}\n")

# recalibrate and re-evaluate (XXX - all batches must be same length)
detector.step(batch3)
print(f"\nState after new batch, recalibration: {detector.state}\n")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w


State after initial batch: baseline



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w


State after test batch: alarm



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w


State after new batch, recalibration: alarm



## Final Notes

We can see the baseline state after processing the initial batch, an alarm raised after observing test data, and then another alarm signal after a new test batch is observed and the reference is internally recalibrated.